# 1. Objetivo
O objetivo deste notebook é reproduzir a metodologia do cálculo de consumo aparente, apresentado no artigo [Indicadores de Consumo Aparente de Bens Industriais: metodologia e resultados](http://www.ipeadata.gov.br/doc/td_2101.pdf).

**Autor:** Amilton Lobo Mendes Júnior
amilton.mendes@gmail.com

# 2. Preparação do ambiente


## 2.1 Instalação de bibliotecas utilizadas

In [71]:
!pip install pyIpeaData

##2.2. Import das bibliotecas utilizadas neste notebook





In [72]:
import pandas as pd
import requests
import json
from pandas import json_normalize
from datetime import datetime
import pyIpeaData as ip

## 2.3 URL das tabelas de recursos e usos de 2021, armazenada no github.

In [73]:
url_recursos = 'https://github.com/amiltonmendes/consumo-aparente/raw/refs/heads/main/MIP/68_tab1_2021.xls'
url_usos = 'https://github.com/amiltonmendes/consumo-aparente/raw/refs/heads/main/MIP/68_tab2_2021.xls'


## 2.4 Dicionários com informações sobre os dados por setor

In [74]:
#Criar um mapa que faça a correlação entre um código de cnae e códigos para baixar a série de importação, exportação e a url para download de dados do sidra

mapa_cnae = {
    "2100": {  # CNAE 2100 - Fabricação de produtos farmacêuticos
        "importacao": "FUNCEX12_MQFARM2N12",
        "exportacao": "FUNCEX12_XQFARM2N12",
        "sidra_url": "https://apisidra.ibge.gov.br/values/t/8888/n1/all/v/12606/p/all/c544/129330/d/v12606%205"
    },
    # adicione mais CNAEs aqui...
}



## 2.5 Funções e classes utilizadas no notebook

In [ ]:
producao['Código do produto'].str.startswith(cnae)][['Total\ndo produto']].values[0][0]

In [ ]:
#Carrega dados oferta
def carrega_dados_oferta():
  oferta = pd.read_excel(url_recursos, sheet_name='oferta',skiprows=3,skipfooter=2,dtype={'Unnamed: 0' : str})
  #Renomear as duas primeiras colunas que não foram carregadas por estarem mescladas
  oferta= oferta.rename(columns={oferta.columns[0] : 'Código do produto', oferta.columns[1] : 'Descrição do produto'})
  #Remover a primeira linha (vazia)
  oferta = oferta.iloc[1:]
  return oferta
def carrega_dados_producao():
  producao = pd.read_excel(url_recursos, sheet_name='producao',skiprows=3,skipfooter=2,dtype={'Unnamed: 0' : str})
  producao = producao.rename(columns={producao.columns[0] : 'Código do produto', producao.columns[1] : 'Descrição do produto'})
  producao = producao.iloc[1:]
  return producao

def carrega_dados_importacao():
  importacao = pd.read_excel(url_recursos, sheet_name='importacao',skiprows=3,skipfooter=5,dtype={'Unnamed: 0' : str})
  importacao = importacao.rename(columns={importacao.columns[0] : 'Código do produto', importacao.columns[1] : 'Descrição do produto'})
  importacao = importacao.iloc[1:]
  return importacao

def carrega_dados_exportacao():
  exportacao = pd.read_excel(url_usos, sheet_name='demanda',skiprows=3,skipfooter=2,dtype={'Unnamed: 0' : str})
  exportacao = exportacao.rename(columns={exportacao.columns[0] : 'Código do produto', exportacao.columns[1] : 'Descrição do produto'})
  exportacao = exportacao.iloc[1:]
  return exportacao

class ConsumoAparente():
  def __init__(self,oferta,producao,importacao,exportacao):
    self.oferta = oferta
    self.producao = producao
    self.importacao = importacao
    self.exportacao = exportacao
  def carrega_dados_producao_mensal(self,url):
    r = requests.get(url)
    #Apresentar data de carregamento
    #st.write(f"Data de carregamento: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
    a = json.loads(r.text)
    res = json_normalize(a)

    producao = pd.DataFrame(res)
    nomes_colunas = producao.iloc[0].values
    producao.columns = nomes_colunas
    producao = producao.iloc[1:]
    producao['Mês (Código)'] = producao['Mês (Código)'].astype(str)
    producao['Ano'] = producao['Mês (Código)'].str[:4]
    producao['Mes'] = producao['Mês (Código)'].str[4:]

    producao['Ano'] = producao['Ano'].astype(int)
    producao['Mes'] = producao['Mes'].astype(int)

    producao = producao[['Ano','Mes','Valor']]
    producao = producao.rename(columns={'Valor' : 'producao'})

    producao['producao'] = producao['producao'].astype(float)

    return producao.copy()
  def carrega_dados_quantum_exp_imp(self,url,movimento):
    ##Carrega dados de quantum de exportação e importação
    dados_exportacao_farmacos = ip.get_serie(mapa_cnae[cnae]['exportacao'])
    dados_importacao_farmacos = ip.get_serie(mapa_cnae[cnae]['importacao'])

    dados_exportacao_farmacos['VALDATA'] = pd.to_datetime(dados_exportacao_farmacos['VALDATA'],utc=True)
    dados_importacao_farmacos['VALDATA'] = pd.to_datetime(dados_importacao_farmacos['VALDATA'],utc=True)

    dados_exportacao_farmacos['Ano'] = dados_exportacao_farmacos['VALDATA'].dt.year
    dados_importacao_farmacos['Ano'] = dados_importacao_farmacos['VALDATA'].dt.year
    dados_exportacao_farmacos['Mes'] = dados_exportacao_farmacos['VALDATA'].dt.month
    dados_importacao_farmacos['Mes'] = dados_importacao_farmacos['VALDATA'].dt.month

    dados_exportacao_farmacos = dados_exportacao_farmacos[['Ano','Mes','VALVALOR']]
    dados_importacao_farmacos = dados_importacao_farmacos[['Ano','Mes','VALVALOR']]

    dados_exportacao_farmacos = dados_exportacao_farmacos.rename(columns={'VALVALOR' : 'exportacao'})
    dados_importacao_farmacos = dados_importacao_farmacos.rename(columns={'VALVALOR' : 'importacao'})

    dados_exportacao_farmacos['exportacao'] = dados_exportacao_farmacos['exportacao'].astype(float)
    dados_importacao_farmacos['importacao'] = dados_importacao_farmacos['importacao'].astype(float)

  def calcula_consumo_aparente(self,cnae,verbose=0):
    preco_consumidor,preco_basico = self.oferta[self.oferta['Código do produto'].str.startswith(cnae)][['Oferta total\na preço\nde consumidor','Oferta total\n a preço básico ']].values[0]
    fator_ajuste = preco_basico/preco_consumidor
    valor_producao = self.producao[self.producao['Código do produto'].str.startswith(cnae)][['Total\ndo produto']].values[0][0]
    valor_exportacao = self.exportacao[self.exportacao['Código do produto'].str.startswith(cnae)][['Exportação\nde bens e\nserviços (1)']].values[0][0]
    valor_exportacao = valor_exportacao*fator_ajuste
    valor_importacao = self.importacao[self.importacao['Código do produto'].str.startswith(cnae)]['Importação de bens\ne serviços (1)'].values[0]
    #Calcular pesos de acordo com fórmula constante do arquivo
    total = valor_importacao - valor_exportacao + valor_producao
    peso_importacao = valor_importacao/total
    peso_exportacao = valor_exportacao/total
    peso_producao = valor_producao/total

    #Prepara dados de quantum de exportação e importação para a junção com dados de produção e cálculo de consumo aparente
    dados

    producao = self.carrega_dados_producao_mensal(mapa_cnae[cnae]['sidra_url'])



    producao = producao.merge(dados_importacao_farmacos,how='inner',on=['Ano','Mes']).merge(dados_exportacao_farmacos,how='inner',on=['Ano','Mes'])

    producao['producao'] = producao['producao'].astype(float)
    producao['importacao'] = producao['importacao'].astype(float)
    producao['exportacao'] = producao['exportacao'].astype(float)

    producao['consumo_aparente'] = producao['producao']*peso_producao - producao['exportacao']*peso_exportacao + producao['importacao']*peso_importacao





Index(['Unnamed: 0', 'Unnamed: 1', 'Oferta total\na preço\nde consumidor',
       'Margem \nde\ncomércio', 'Margem\nde\ntransporte',
       'Imposto\nde\nimportação', 'IPI', 'ICMS',
       'Outros impostos menos subsídios',
       'Total de impostos líquidos de subsídios',
       'Oferta total\n a preço básico '],
      dtype='object')


,Código do produto,Descrição do produto,Oferta total\na preço\nde consumidor,Margem \nde\ncomércio,Margem\nde\ntransporte,Imposto\nde\nimportação,IPI,ICMS,Outros impostos menos subsídios,Total de impostos líquidos de subsídios,Oferta total\n a preço básico
1,01911,"Arroz, trigo e outros cereais",50589.0,4691.0,1521.0,2.0,0.0,68.0,550.0,620.0,43757.0
2,01912,Milho em grão,125926.0,12347.0,3147.0,0.0,0.0,471.0,409.0,880.0,109552.0
3,01913,"Algodão herbáceo, outras fibras da lav. tempor...",34790.0,2068.0,436.0,3.0,0.0,90.0,318.0,411.0,31875.0


In [ ]:



url = 'https://apisidra.ibge.gov.br/values/t/8888/n1/all/v/12606/p/all/c544/129330/d/v12606%205'
r = requests.get(url)
#Apresentar data de carregamento
#st.write(f"Data de carregamento: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
a = json.loads(r.text)
res = json_normalize(a)

producao = pd.DataFrame(res)




In [ ]:
nomes_colunas = producao.iloc[0].values
producao.columns = nomes_colunas
producao = producao.iloc[1:]
producao = producao.copy()

In [ ]:
total = valor_importacao - valor_exportacao + valor_producao
peso_importacao = valor_importacao/total
peso_exportacao = valor_exportacao/total
peso_producao = valor_producao/total

In [ ]:
print(f'Peso da importação: {peso_importacao}')
print(f'Peso da exportação: {peso_exportacao}')
print(f'Peso da produção: {peso_producao}')

Peso da importação: 0.42250981749526956
Peso da exportação: 0.02206616206135398
Peso da produção: 0.5995563445660844


In [27]:


dados_exportacao_farmacos.head(3)

,Ano,Mes,exportacao
0,1997,1,22.91
1,1997,2,15.84
2,1997,3,16.23


In [39]:
dados_importacao_farmacos.head(3)

,Ano,Mes,importacao
0,1997,1,12.44
1,1997,2,16.00
2,1997,3,23.49


In [ ]:
!pip install sidrapy

In [60]:

producao['Mês (Código)'] = producao['Mês (Código)'].astype(str)
producao['Ano'] = producao['Mês (Código)'].str[:4]
producao['Mes'] = producao['Mês (Código)'].str[4:]

producao['Ano'] = producao['Ano'].astype(int)
producao['Mes'] = producao['Mes'].astype(int)

producao = producao[['Ano','Mes','Valor']]
producao = producao.rename(columns={'Valor' : 'producao'})

producao.head(3)

,Ano,Mes,producao
1,2002,1,55.42792
2,2002,2,76.43190
3,2002,3,86.01877


In [61]:

producao['Mês (Código)'] = producao['Mês (Código)'].astype(str)
producao['Ano'] = producao['Mês (Código)'].str[:4]
producao['Mes'] = producao['Mês (Código)'].str[4:]

producao['Ano'] = producao['Ano'].astype(int)
producao['Mes'] = producao['Mes'].astype(int)

producao = producao[['Ano','Mes','Valor']]
producao = producao.rename(columns={'Valor' : 'producao'})

producao['producao'] = producao['producao'].astype(float)
producao['producao'] = producao['producao'].round(2)
producao.head(3)


,Ano,Mes,producao
1,2002,1,55.43
2,2002,2,76.43
3,2002,3,86.02


In [62]:
producao['Mês (Código)'] = producao['Mês (Código)'].astype(str)
producao['Ano'] = producao['Mês (Código)'].str[:4]
producao['Mes'] = producao['Mês (Código)'].str[4:]

producao['Ano'] = producao['Ano'].astype(int)
producao['Mes'] = producao['Mes'].astype(int)

producao = producao[['Ano','Mes','Valor']]
producao = producao.rename(columns={'Valor' : 'producao'})

producao['producao'] = producao['producao'].astype(float)
producao['producao'] = producao['producao'].round(2)


producao = producao.merge(dados_importacao_farmacos,how='inner',on=['Ano','Mes']).merge(dados_exportacao_farmacos,how='inner',on=['Ano','Mes'])

producao['producao'] = producao['producao'].astype(float)
producao['importacao'] = producao['importacao'].astype(float)
producao['exportacao'] = producao['exportacao'].astype(float)

producao['consumo_aparente'] = producao['producao']*peso_producao - producao['exportacao']*peso_exportacao + producao['importacao']*peso_importacao



In [63]:
producao['consumo_aparente_arredondado'] = producao['consumo_aparente'].round(2)
producao.head(3)

,Ano,Mes,producao,importacao,exportacao,consumo_aparente,consumo_aparente_arredondado
0,2002,1,55.43,32.93,19.73,46.6940,46.69
1,2002,2,76.43,29.31,18.53,57.7976,57.80
2,2002,3,86.02,40.93,21.86,68.3654,68.37


In [66]:
#calcular a variação do ano anterior, ou seja, 12 meses antes do mês de referência
producao['consumo_aparente_12_meses_arredondado'] = producao['consumo_aparente_arredondado'].shift(12)
producao['variacao_consumo_aparente_arredondado'] = ((producao['consumo_aparente_arredondado']/producao['consumo_aparente_12_meses_arredondado'])-1)*100
producao.tail(13)

,Ano,Mes,producao,importacao,exportacao,consumo_aparente,consumo_aparente_arredondado,consumo_aparente_12_meses_arredondado,variacao_consumo_aparente_arredondado
264,2024,1,72.82,148.36,80.55,104.3922,104.39,129.24,-19.227793
265,2024,2,82.38,139.93,70.39,106.7908,106.79,120.42,-11.318718
266,2024,3,92.89,144.37,88.47,114.6000,114.60,152.32,-24.763655
267,2024,4,100.30,143.80,107.53,118.4254,118.43,137.56,-13.906659
268,2024,5,103.72,182.93,102.00,137.0226,137.02,127.12,7.787917
269,2024,6,107.13,147.72,92.16,124.4772,124.48,117.25,6.166311
270,2024,7,102.95,182.17,107.38,136.1338,136.13,116.84,16.509757
271,2024,8,110.05,164.84,91.88,133.4252,133.43,132.57,0.648714
272,2024,9,102.27,145.04,107.38,120.1312,120.13,114.95,4.506307
273,2024,10,109.17,167.95,104.73,133.9464,133.95,129.53,3.412337
